In [90]:
import pandas as pd
import numpy as np
import numpy_financial as npf
import random
import math
import datetime as dt
import arrow


In [184]:
from __future__ import annotations

import math

import datetime as dt
import numpy_financial as npf
import pandas as pd


class Loan:
    """Loan class to create instances of installment loans.
     To be initialized with the following parameters:
        1. loan_amount:float - The Original loan amount (principal) disbursed
        on the loan date.
        2. int_rate:float - original rate of interest applicable on the
        principal outstanding.
        3. fees:float - Origination fees charged at the time of booking,
        expressed as a % of original loan amount.
        4. term:float - The original term of the loan per schedule.
        5. segment:str - The approx risk category of the loan. Broadly mapped
        to six FICO_score groups. Configurable via config.yaml for categories'
        6. channel:str - Indicator variable to identify if the loan was booked
        through a free channel or a paid channel.
        7. loan_dt:str - Date of loan disbursement.
        8. freq:str - Frequency of repayment. Monthly, Quarterly etc. Valid
        values can be accessed via the class variable loan.valid_pmt_freq
        """

    valid_pmt_freq = {
        'W': 'Weekly payments', '2W': 'Fortnightly payments',
        'M': 'Monthly payments', 'BM': 'Bi-monthly payments',
        'Q': 'Quarterly payments', 'H': 'Semi-annual payments',
        'Y': 'Annual payments',
    }
    freq_offset = {
        'W': pd.DateOffset(days=7, months=0),
        '2W': pd.DateOffset(days=14, months=0),
        'M': pd.DateOffset(days=0, months=1),
        'BM': pd.DateOffset(days=0, months=2),
        'Q': 'Q', 'H': '2Q', 'Y': 'Y',
    }
    period_to_months = {
        'W': 7.0 / 30, '2W': 14.0 / 30,
        'M': 1, 'BM': 2, 'Q': 3, 'H': 6, 'Y': 12,
    }

    def __init__(
            self, loan_amt: float, interest_rate: float, term_in_months: float,
            loan_dt:
            str,
            freq: str = 'M', fees_pct: float = 0.0,
            segment: str = 'c', channel: str = 'free',
    ):
        self.loan_amt = loan_amt
        self.interest_rate = interest_rate
        self.fees_pct = fees_pct
        self.term_in_months = term_in_months
        self.segment = segment
        self.channel = channel
        self.loan_dt = dt.datetime.strptime(loan_dt, '%Y-%m-%d')
        self.freq = freq
        self._offset = self.freq_offset[self.freq]
        self._periods = math.ceil(
            self.term_in_months / self.period_to_months[self.freq]
        )
        self._period_interest_rate = self.interest_rate * self \
            .period_to_months[self.freq] / 12
        self.pmt = -npf.pmt(
            self._period_interest_rate,
            self._periods, self.loan_amt,
        )

    def get_cfsch(self) -> pd.DataFrame:
        """Method to get the original scheduled of cashflows for a given loan.
        For monthly frequency (most common), it assumes that the dues date are
        on the same day of the month every month.
        Usage: loan.get_schedule()"""
        df = pd.DataFrame()
        df['dates'] = pd.Series(
            pd.date_range(
                self.loan_dt, freq=self._offset, periods=self._periods + 1,
            ),
        ).shift(-1).dropna()
        df['period'] = df.index + 1
        df['interest_pmt'] = - \
            npf.ipmt(
                self._period_interest_rate,
                df['period'], self._periods, self.loan_amt,
            )
        df['principal_pmt'] = - \
            npf.ppmt(
                self._period_interest_rate,
                df['period'], self._periods, self.loan_amt,
            )
        df['closing_principal'] = self.loan_amt - df['principal_pmt'].cumsum()
        df['opening_principal'] = df['closing_principal'].shift(
            1,
        ).fillna(self.loan_amt)
        df = df[['dates', 'period', 'opening_principal', 'interest_pmt',
                 'principal_pmt', 'closing_principal']]
        return df

    @property
    def wal(self) -> float:
        """Returns the weighted average life of the loan (in months) based on
        the original cashflow schedule.
        The [WAL](https://en.wikipedia.org/wiki/Weighted-average_life) of the
        loan can be defined as the average number of months it takes for the
        principal of the loan
        to be repaid, if the borrower repays by the original schedule."""
        _cfs = self.get_cfsch()
        return (_cfs['principal_pmt'] * _cfs['period']).sum() * \
            self.period_to_months[self.freq] / self.loan_amt

    @property
    def apr(self) -> float:
        """Returns the Annual percentage rate (APR) of the loan based on the
        original cashflow schedule.
        The [APR](https://en.wikipedia.org/wiki/Annual_percentage_rate) of the
        loan can be defined as the
        total financial cost of the loan (including fees) divided by the WAL of
        the loan."""
        return self.interest_rate + (self.fees_pct / (self.wal / 12))

    def get_modcfs(self, addl_pmts: dict) -> None:
        cols = ['dates', 'period', 'opening_principal',
                'opening_accrued_interest',
                'current_period_interest', 'interest_pmt', 'principal_pmt',
                'additional_pmt', 'total_pmt', 'closing_principal']
        df = self.get_cfsch()
        df['additional_pmt'] = pd.Series(addl_pmts,
                                         index=df.index).fillna(0)
        df['opening_accrued_interest'] = 0
        df['current_period_interest'] = 0
        df['closing_accrued_interest'] = 0
        df['total_pmt']  = 0
        cl_p = self.loan_amt
        cl_ai = 0   # opening accrued interest
        for idx, row in df.iterrows():
            row.loc['opening_principal'] = cl_p
            row.loc['opening_accrued_interest'] = cl_ai
            row.loc['current_period_interest'] = \
                row.loc['opening_principal'] * self._period_interest_rate
            row.loc['total_pmt'] = min(row.loc['interest_pmt'] +
                                       row.loc['principal_pmt'] +
                                       row.loc['additional_pmt'],
                                       row.loc['opening_principal'] +
                                       row.loc['opening_accrued_interest'] +
                                       row.loc['current_period_interest'],
                                       )
            row.loc['closing_accrued_interest'] = \
                max(0,
                    row.loc['opening_accrued_interest'] +
                    row.loc['current_period_interest'] -
                    row.loc['total_pmt'],
                    )
            cl_ai = row.loc['closing_accrued_interest']
            row.loc['closing_principal'] = \
                max(0,
                    row.loc['opening_principal'] +
                    row.loc['opening_accrued_interest'] +
                    row.loc['current_period_interest'] -
                    row.loc['total_pmt'],
                    )
            cl_p = row.loc['closing_principal']
            df.loc[idx, cols] = row.loc[cols]
        return df[[col for col in cols if col not in ['interest_pmt', 
                                                      'principal_pmt']]]



In [185]:
ir, t, fr, fe = (0.0599, 36, 'M', 0.05)
l1 = Loan(loan_amt=10000, interest_rate=ir, term_in_months=t, loan_dt='2022-12-12',
                      freq=fr, fees_pct=fe)

In [186]:
l1.get_cfsch().style.format('${:,.2f}')

,dates,period,opening_principal,interest_pmt,principal_pmt,closing_principal
0,"$,.2f",$1.00,"$10,000.00",$49.92,$254.26,"$9,745.74"
1,"$,.2f",$2.00,"$9,745.74",$48.65,$255.53,"$9,490.22"
2,"$,.2f",$3.00,"$9,490.22",$47.37,$256.80,"$9,233.41"
3,"$,.2f",$4.00,"$9,233.41",$46.09,$258.08,"$8,975.33"
4,"$,.2f",$5.00,"$8,975.33",$44.80,$259.37,"$8,715.96"
5,"$,.2f",$6.00,"$8,715.96",$43.51,$260.67,"$8,455.29"
6,"$,.2f",$7.00,"$8,455.29",$42.21,$261.97,"$8,193.32"
7,"$,.2f",$8.00,"$8,193.32",$40.90,$263.28,"$7,930.05"
8,"$,.2f",$9.00,"$7,930.05",$39.58,$264.59,"$7,665.46"
9,"$,.2f",$10.00,"$7,665.46",$38.26,$265.91,"$7,399.55"


In [187]:
d = {0:500, 10:200}
df = l1.get_modcfs(d)
df
# df = l1.get_cfsch()
# for idx, row in df.iterrows():
#     #print(idx, row)
#     print(row.principal_pmt)
#     print(row.loc['principal_pmt'])

,dates,period,opening_principal,opening_accrued_interest,current_period_interest,additional_pmt,total_pmt,closing_principal
0,2023-01-12,1,10000.000000,0,49.916667,500.0,804.174066,9245.742600
1,2023-02-12,2,9245.742600,0,46.151665,0.0,304.174066,8987.720199
2,2023-03-12,3,8987.720199,0,44.863703,0.0,304.174066,8728.409836
3,2023-04-12,4,8728.409836,0,43.569312,0.0,304.174066,8467.805082
4,2023-05-12,5,8467.805082,0,42.268460,0.0,304.174066,8205.899476
5,2023-06-12,6,8205.899476,0,40.961115,0.0,304.174066,7942.686524
6,2023-07-12,7,7942.686524,0,39.647244,0.0,304.174066,7678.159701
7,2023-08-12,8,7678.159701,0,38.326814,0.0,304.174066,7412.312449
8,2023-09-12,9,7412.312449,0,36.999793,0.0,304.174066,7145.138175
9,2023-10-12,10,7145.138175,0,35.666148,0.0,304.174066,6876.630257


In [ ]:
channel_prop = {'free':0.8, 'paid':0.2}
channel_cost = {'free':0.0, 'paid':0.05}
term_prop = {'36':0.7, '60':0.3}
term_prem = {'36':0.0, '60':0.01} #reference term premium from treasury site
fee = {'No Fees': 0, '5% fees': 0.05, '10% fees': 0.1}

segment_ScoreMapping = {'f':'FICO:300-550', 'e':'FICO:550-650', 'd':'FICO:650-699',
                'c':'FICO:700-749', 'b':'FICO:750-799', 'a':'FICO:800-850'}

segment_prop = {'f':{'300-550':8.4%}, 'e':{'550-650':16.3%}, 'd':{'650-699':12.5%},
                'c':{'700-749':16.4%}, 'b':{'750-799':23.1%}, 'a':{'800-850':23.3%}} #six segments tied to chargeoffs and prepayment rates
chOff_mean = {'a':1%, 'b':1%, 'c':4.4%, 'd':8.9%, 'e':mean(22.5%, 15.8%), 'f':mean(28.4%, 41%)} # researchgate reference from 2000-2002
#establish a relationship between chargeoff and prepayment rates or FICO and prepayment rates
prepay_mean = {'a':45%, 'b':40%, 'c':35%, 'd':30%, 'e':25%, 'f':20%} # purely arbitrary


In [9]:
#Generating the prepayment curve and the Loss curve

# Losses peak by 9 months an36 std of about 3 months for a 36 month loan = 9/36 = 0.25; var = 9 mthsq = 9/36^2 = 0.00695 
# prepayments peak by 12 months and std by 6 months for a 36 month loan = 12/36 = 0.33; var = 9 mthsq = 36/36^2 = 0.0278

# alpha = mean*((mean*(1-mean)/var)-1)
# beta = (1-mean)*((mean*(1-mean)/var)-1)

# alpha_co = 0.25*((0.25*(1-0.25)/0.00695)-1)
# beta_co = (1-0.25)*((0.25*(1-0.25)/0.00695)-1)

# alpha_ppay = 0.33*((0.33*(1-0.33)/0.0278)-1)
# beta_ppay = (1-0.33)*((0.33*(1-0.33)/0.0278)-1)


In [22]:
# #sampling from a beta distribution
# x = math.floor(36*random.betavariate(alpha_co, beta_co))
# x

10

In [36]:
# y = math.floor(36*random.betavariate(alpha_ppay, beta_ppay))
# y

7